# Import all the stuff

In [1]:
import random
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from keras import backend as K
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.optimizers import SGD
from keras.initializers import RandomUniform

from bert_serving.client import BertClient

Using TensorFlow backend.


# Testing BertClient - be sure to have the server running

In [2]:
bc = BertClient()
bc.encode(['First do it', 'then do it right', 'then do it better'])

array([[ 0.13186474,  0.32404128, -0.82704437, ..., -0.3711958 ,
        -0.39250174, -0.31721866],
       [ 0.24873531, -0.12334418, -0.3893385 , ..., -0.44756195,
        -0.55913526, -0.11345161],
       [ 0.28627345, -0.18580122, -0.30906814, ..., -0.2959366 ,
        -0.39310536,  0.07640187]], dtype=float32)

# Testing Keras

In [3]:
x_input = np.array([[1,2,3,4,5]])
y_input = np.array([[10]])

model = Sequential()
model.add(Dense(units=32, activation="tanh", input_dim=x_input.shape[1], kernel_initializer='random_normal'))
model.add(Dense(units=1, kernel_initializer='random_normal'))

model.compile(loss='mse', optimizer='sgd', metrics=['accuracy'])

model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                192       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 225
Trainable params: 225
Non-trainable params: 0
_________________________________________________________________


# Load the commit data

In [29]:
f = open("data/rubygems-raw-commits.txt", "r")
# Skip the first line
for i in range(4):
    f.readline()

# This is the first commit
commit_message = f.readline()[4:] # Removing the tab on all the commit
print("Commit:", commit_message)

commits_list = [commit_message]

# Build the other commit list
number_of_commit = 2000
for i in range(number_of_commit):
    line = f.readline()
    while "commit" not in line: 
        line = f.readline()

    for i in range(3):
        f.readline()

    # This is the first commit
    commit_message = f.readline()[4:]
    
    if commit_message != '' and "Merge pull request" not in commit_message and commit_message != '\n' and "+0" not in commit_message and "it " not in commit_message and "e: " not in commit_message:
        print("Commit:", commit_message)
        commits_list.append(commit_message)

Commit: Pass `--debug` flag with JRUBY_OPTS

Commit: Set TRUFFLERUBYOPT only when needed

Commit: Skip test that fails on jruby in debug mode

Commit: Fix `gem outdated` incorrectly handling platform specific gems

Commit: causing this bug.

Commit: Remove weird comment

Commit: Remove unnecessary method

Commit: These are not needed

Commit: Reduce the number of CI jobs

Commit: Make tests independent from the major version

Commit: Remove legacy stuff from a long time ago

Commit: Apply hack to ruby 2.3 as well

Commit: Test truffleruby 20.3

Commit: Bump tested rubygems to latest patch levels

Commit: Fix performance regression in resolver

Commit: Remove unnecessary interpolation

Commit: Add ruby 3.0.0 and rubygems 3.2.4 to CI

Commit: Quote ruby version names in YAML files

Commit: Add a missing space

Commit: Bump minitest dependency to 5.14.3

Commit: Always bundle dev gems using HEAD bundler

Commit: Make sure `universal.x86_64` cpus are detected as x86_64

Commit: Infer local

Commit: Prefix bundler release tags with `bundler-`

Commit: Adapt release scripts to new branch convention

Commit: Update versioning used in release playbook to something current

Commit: Use `rake` binstub for releasing too

Commit: Base bundler releases off rubygems stable branches

Commit: Consistently use `Bundler::GemHelper.gemspec`

Commit: Move build metadata tasks to release file

Commit: Tag the specs with :readline

Commit: Fix method no longer defined

Commit: Extract a local variable

Commit: Show progress

Commit: Use a bundler specific release branch name

Commit: Normalize stable branch resolution

Commit: Filter out only commits touching bundler's code

Commit: Use the new repo in URLs

Commit: Remove task that our specs already check

Commit: Add tests with different flag combinations

Commit: Allow setting a tag prefix to be used by gem release tasks

Commit: Fix line spacing to make Layout/EmptyLines happy

Commit: This also fixes the spec w.r.t change in the task


# Run the main modal

In [30]:
bert_encode_commits = bc.encode(commits_list)
print(bert_encode_commits)

[[ 0.11776623 -0.37144774  0.5724507  ... -0.48307884 -0.12252332
   0.54817104]
 [-0.47575814 -0.2392094   0.38235873 ... -0.10153607 -0.59130967
   0.364713  ]
 [-0.7100193  -0.923654    0.35236982 ... -0.29477942 -0.1532885
   0.31129095]
 ...
 [-0.15253721 -0.5510403   0.12561513 ... -0.5397371  -0.2651676
   0.4117009 ]
 [-0.34680808 -0.29943636  0.20318459 ... -0.6690542   0.08993843
   0.05774815]
 [-0.06430899 -0.36786002  0.00188022 ... -0.4379144  -0.09372777
   0.02485875]]


In [43]:
x_input = np.array(bert_encode_commits)
y_input = np.ones((len(x_input),1))

model = Sequential()
model.add(Dense(units=32, activation="sigmoid", input_dim=x_input.shape[1]))
model.add(Dense(units=8, activation="relu", input_dim=x_input.shape[1]))
model.add(Dense(units=1))

model.compile(loss='mse', optimizer='sgd', metrics=['accuracy'])

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 32)                24608     
_________________________________________________________________
dense_17 (Dense)             (None, 8)                 264       
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 9         
Total params: 24,881
Trainable params: 24,881
Non-trainable params: 0
_________________________________________________________________


In [44]:
history = model.fit(x_input, y_input, epochs=40, batch_size=len(x_input))

Epoch 1/40
1285/1285 [==============================] - 0s 76us/step - loss: 0.0801 - accuracy: 0.9673
Epoch 2/40
1285/1285 [==============================] - 0s 4us/step - loss: 0.0305 - accuracy: 0.9984
Epoch 3/40
1285/1285 [==============================] - 0s 5us/step - loss: 0.0191 - accuracy: 0.9992
Epoch 4/40
1285/1285 [==============================] - 0s 4us/step - loss: 0.0164 - accuracy: 0.9992
Epoch 5/40
1285/1285 [==============================] - 0s 5us/step - loss: 0.0156 - accuracy: 0.9992
Epoch 6/40
1285/1285 [==============================] - 0s 5us/step - loss: 0.0154 - accuracy: 0.9992
Epoch 7/40
1285/1285 [==============================] - 0s 4us/step - loss: 0.0152 - accuracy: 0.9992
Epoch 8/40
1285/1285 [==============================] - 0s 5us/step - loss: 0.0151 - accuracy: 0.9992
Epoch 9/40
1285/1285 [==============================] - 0s 4us/step - loss: 0.0150 - accuracy: 0.9992
Epoch 10/40
1285/1285 [==============================] - 0s 5us/step - loss: 0.01

In [45]:
model.predict(x_input, batch_size=len(x_input))

array([[1.0872725 ],
       [1.1218847 ],
       [1.038754  ],
       ...,
       [0.9535169 ],
       [0.93572265],
       [0.9047688 ]], dtype=float32)

In [46]:
test_commits_list = ["Adding new stuff", "this is a normal commit", "22232", "wtf"]
bert_encode_test_commits = bc.encode(test_commits_list)
print(bert_encode_test_commits)

model.predict(bert_encode_test_commits)

[[ 0.13072483 -0.30913815 -0.19770074 ... -0.0454914  -0.28111756
  -0.16368099]
 [-0.4317877  -0.45127526  0.07794844 ... -0.23766913 -0.05393345
   0.80593836]
 [ 0.16380844 -0.516343    0.13188474 ... -0.25351387 -0.04303393
   0.4613328 ]
 [-0.10335744  0.00934411 -0.27467698 ...  0.01205477  0.02043476
   0.06125272]]


array([[0.84128755],
       [0.9109139 ],
       [1.0522897 ],
       [0.91592366]], dtype=float32)